# Chat Completion API

In [ ]:
!pip install openai

## 환경설정
1. openai 사이트에서 API KEY 발급
2. google colab - secret 탭에 `OPENAI_API_KEY` 등록

In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕~ 내 이름은 윤두준이야~"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕하세요, 윤두준님! 만나서 반가워요. 어떻게 도와드릴까요?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력/예시
4. Output Indicator 출력 지시

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스 AFP 속보 시스템에서 도입되어 사용

In [ ]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요'
title_before = '졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조 등에 있어서 교정 작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사 제목을 읽고 주요 내용을 이해하세요.
2. 제목이 전달하고자 하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성 등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈 수 있는지 점검하고 매우 간결하게 정리하세요.

### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ###
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

### Extra Instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사 제목을 간결히 수정해 주세요.

{title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: "졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환"
- 교정 제목: "과중한 작업으로 끼니를 놓치는 노동자들의 고충"


In [ ]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조 등에 있어서 교정 작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사 제목을 읽고 주요 내용을 이해하세요.
    2. 제목이 전달하고자 하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성 등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈 수 있는지 점검하고 매우 간결하게 정리하세요.

    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사 제목을 간결히 수정해 주세요.

    {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
            "role": "system",
            "content": [
                {
                "type": "text",
                "text": system_message
                }
            ]
            },
            {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": user_message
                }
            ]
            }
        ],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길~'
output = correct_news_title(title_before)
print(output)

- 원래 제목: "주말 미친 폭우 예상, 모두들 무사하시길~"
- 교정 제목: "주말 폭우 예보, 안전 유의하세요"


## 연애 코치 ReAct

In [3]:
# 함수 리팩토링

def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애 코치로써 사용자의 고민을 해결해 주세요.

    << Output Format >>
    1. 상황 분석:

    2. 행동 계획:

    3. 실행:
    """

    user_message = f"""
    사용자 현재 상황:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
            "role": "system",
            "content": [
                {
                "type": "text",
                "text": system_message
                }
            ]
            },
            {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": user_message
                }
            ]
            }
        ],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [8]:
print(dating_coach('저 이번주말에 여자친구와 100일인데, 기억에 남을 선물을 하고 싶어요. 뭐가 좋을까요?'))

1. 상황 분석:
   - 현재 사용자와 여자친구는 100일이라는 특별한 기념일을 맞이하고 있습니다.
   - 사용자는 기억에 남을 선물을 고민하고 있으며, 이를 통해 여자친구에게 특별한 감정을 표현하고 싶어합니다.
   - 선물이 여자친구의 취향이나 두 사람의 관계를 반영하는 것이 중요합니다.

2. 행동 계획:
   - 여자친구의 취향, 관심사, 원하는 것 등을 고려하여 선물을 선정합니다.
   - 선물과 함께 직접 만든 편지나 카드 등 감동적인 메시지를 추가하여 선물의 가치를 높입니다.
   - 기념일에 맞춰 특별한 데이트 코스나 장소를 계획합니다.

3. 실행:
   - 다양한 선물 아이디어를 brainstorming 합니다. 예를 들어:
     - 맞춤형 쥬얼리(손목시계, 목걸이 등)에 그녀의 이니셜을 새겨 넣기
     - 100일 동안의 추억을 담은 포토북 만들기
     - 그녀가 좋아하는 향수나 뷰티 제품
     - 함께할 수 있는 액티비티(스파, 요리 수업, 여행 등)의 예약
   - 선물과 함께 정성을 담아 편지나 카드를 작성하여 그녀에게 전달합니다.
   - 주말에 기념일을 축하하기 위해 특별한 장소에서의 저녁 식사나 데이트를 계획하여 완벽한 하루를 보낼 준비를 합니다.


In [10]:
print(dating_coach('제가 참여하려고 하는 프로젝트에 대해 남자친구가 부정적인 의견을 냈어요. 저는 그냥 응원해줬으면 좋겠는데, 그는 그냥 자기 의견이었다고만 합니다. 이 위기를 어떻게 극복해야 할까요?'))

1. 상황 분석:
   
   현재 당신은 프로젝트에 대한 남자친구의 부정적인 의견 때문에 당혹스럽고 실망감을 느끼고 있군요. 그러한 부정적인 피드백은 당신의 자존감이나 동기를 저하시킬 수 있습니다. 남자친구는 자신의 의견을 표현했지만, 당신은 그 대신 응원과 지지를 기대하고 있습니다. 이 상황은 서로의 기대와 의사소통 방식의 차이에서 비롯된 것으로 보입니다.

2. 행동 계획:

   a. 감정 정리하기: 먼저, 당신의 감정을 명확히 정리하세요. 남자친구의 부정적인 의견이 당신에게 어떤 영향을 미쳤는지 생각해보고, 그 감정이 어디서 오는 것인지 이해합니다.

   b. 솔직한 대화 준비하기: 남자친구와의 대화에서 감정에 대한 솔직한 표현을 할 수 있도록 준비합니다. 그에게 부정적인 의견이 당신에게 어떤 감정을 불러일으켰는지, 그리고 지지와 응원의 중요성을 강조해주세요.

   c. 이해와 공감 구하기: 남자친구의 의견을 단순히 부정적인 것으로 받아들이지 말고, 그가 왜 그런 의견을 갖게 되었는지 그 배경을 이해하려고 노력하세요.

3. 실행:

   - 남자친구와 대화할 시간을 정하세요. 편안한 분위기에서 대화를 시작합니다.
   - "내가 이 프로젝트에 대해 열정적으로 생각하고 있는데, 네 의견이 조금 부정적으로 느껴져서 상처를 받았어."라고 먼저 자신의 감정을 솔직하게 표현합니다.
   - 그 다음, "내가 원하는 것은 내가 노력하는 모습을 응원해주는 것인데, 네가 그걸 도와주지 않으면 좀 힘들 것 같아."라고 기대하는 바를 분명히 전달하세요.
   - 마지막으로 "너의 의견도 중요해. 어떤 점에서 우려가 되는지, 나와 공유해줄 수 있을까?"라고 그의 입장을 듣는 것도 잊지 마세요. 이렇게 하면 서로의 감정을 존중하며 더 나은 대화가 이루어질 것입니다.

이 과정을 통해 서로의 기대와 감정을 더 깊이 이해할 수 있을 것입니다. 함께 힘을 모아 서로의 목표를 응원하는 건강한 관계를 만들어 나가세요.


## 냉털 마스터 ReAct
- 사용자는 냉장고에 남아있는 음식 재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지를 조언해준다.
- Reasoning/Action을 끌어낼 수 있는 적절한 프롬프팅을 작성하세요.

In [12]:
# 함수 리팩토링

def fridge_raid_master(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    냉장고에 남은 재료를 활용하여 요리를 제안하세요.

    - 요구사항: 가능한 모든 재료를 사용하되, 메인 요리를 하나 제안하세요. 너무 복잡하거나 시간이 오래 걸리지 않는 요리로 제한합니다.
    - 재료 목록과 사용 가능한 양을 고려하여 적절한 양의 재료를 사용하세요.
    - 요리 방법과 과정은 상세하게 설명하세요. 가능한 경우, 요리 팁이나 추가 팁을 제공하세요.

    # Steps

    1. 주어진 재료 목록과 양을 분석합니다.
    2. 각 재료의 특성을 고려하여 적합한 요리 아이디어를 생성합니다.
    3. 제안된 요리에 필요한 과정과 조리법을 단계별로 상세히 설명합니다.
    4. 제안된 요리의 조리시간과 어려운 점 또는 주의사항 등을 추가 조언으로 제시합니다.

    # Output Format

    단락 형태의 응답으로 제안된 요리를 설명하고, 필요한 재료와 단계별 조리법을 체계적으로 나열합니다.

    # Examples

    - **Input**: 닭고기 200g, 양파 1개, 당근 1개, 감자 2개, 소금, 후추
    - **Output**:
    - **제안 요리**: 닭고기 야채 스튜
    - **재료**: 닭고기 200g, 양파 1개, 당근 1개, 감자 2개, 소금, 후추
    - **조리법**:
        1. 감자와 당근을 적당한 크기로 썰어줍니다.
        2. 냄비에 기름을 두르고 양파를 볶습니다.
        3. 닭고기를 넣고 갈색으로 변할 때까지 볶습니다.
        4. 썰어놓은 감자와 당근을 넣고 함께 볶습니다.
        5. 물을 재료가 잠길 만큼 부어주고 소금과 후추로 간을 합니다.
        6. 약 30분 간 약한 불에서 끓입니다.
        7. 기호에 따라 추가 양념을 넣어 완성합니다.

    # Notes

    - 재료가 부족할 경우 대체 가능한 재료나 방법을 제안하세요.
    - 사용자가 원하는 요리 스타일이나 식사를 염두에 두고 조리법을 제안할 수 있도록 합니다.
    """

    user_message = f"""
    냉장고에 남은 음식재료:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
            "role": "system",
            "content": [
                {
                "type": "text",
                "text": system_message
                }
            ]
            },
            {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": user_message
                }
            ]
            }
        ],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [23]:
from IPython.display import Markdown
user_foods = ['쌀국수면, 사골육수, 양파 1개, 김치, 올리브유, 소금, 후추, 삼겹살, 쌈장']
display(Markdown(fridge_raid_master(user_foods)))

- **제안 요리**: 삼겹살 쌀국수 볶음

- **재료**: 
  - 쌀국수면 200g
  - 사골육수 1컵
  - 양파 1개
  - 김치 약간 (원하는 만큼)
  - 올리브유 2큰술
  - 소금 1작은술
  - 후추 약간
  - 삼겹살 200g
  - 쌈장 (서브용)

- **조리법**:
  1. **준비**: 쌀국수면을 큰 그릇에 담고 뜨거운 물에 10분 정도 불려서 부드럽게 만들어 주세요. 불린 쌀국수면은 체에 밭쳐 물기를 빼놓습니다.

  2. **재료 준비**: 양파는 얇게 슬라이스하고, 김치는 적당한 크기로 자릅니다.

  3. **삼겹살 구이**: 팬에 올리브유를 두르고 중불로 가열한 후 삼겹살을 넣고 바삭하게 구워줍니다. 양면이 노릇노릇하게 익으면 꺼내서 지방만 제거한 후 먹기 좋은 크기로 잘라줍니다.

  4. **볶기**: 같은 팬에 남은 기름에 양파를 넣고 볶아 부드럽게 익힌 후, 김치를 추가해 볶습니다. 김치의 매운맛이 올라오면 준비한 쌀국수면을 넣고 반 잘 섞어줍니다.

  5. **육수 추가**: 사골육수를 붓고 간을 맞추기 위해 소금과 후추를 넣습니다. 재료가 익고 육수가 어느 정도 줄어들 때까지 볶아줍니다.

  6. **서빙**: 접시에 볶은 쌀국수와 함께 구운 삼겹살을 올리고, 원하시면 쌈장으로 곁들여 제공합니다.

- **조리시간**: 약 30분 소요

- **어려운 점 또는 주의사항**: 지나치게 오래 볶지 않도록 주의하세요. 쌀국수가 물기를 많이 흡수할 수 있으니, 육수를 많이 추가하는 것이 좋습니다. 추가로 고추가루나 다른 양념을 곁들여 매운맛을 즐길 수 있습니다.

## 면접질문 생성 JSON 출력

In [24]:
# 함수 리팩토링

def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    당신은 머신러닝/딥러닝/LLM/AI서비스 개발의 전문가로써, 해당 분야의 으뜸가는 면접관입니다.
    매번 그룹사의 인재를 발굴하기 위해 면접 질문/모범 답안을 작성하고 있습니다.

    <<지시사항>>
    - 사용자가 제출한 채용 공고의 내용을 바탕으로 예상 면접 질문과 답변을 작성해 주세요.
    - 하드스킬과 소프트스킬 두 개의 섹션으로 나누어 작성해 주세요.
    - 각 스킬별로 질문/답변을 3개씩 만들어 주세요.

    <<출력 형식>>
    출력은 json 형식으로만 반환되어야 합니다.

    {{
        "hard_skill": [
            {{
                "question": "",
                "answer": ""
            }}
        ],
        "soft_skill": [
            {{
                "question": "",
                "answer": ""
            }}
        ]
    }}
    """

    user_message = f"""
    채용 공고:
    {job_posting}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
            "role": "system",
            "content": [
                {
                "type": "text",
                "text": system_message
                }
            ]
            },
            {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": user_message
                }
            ]
            }
        ],
        response_format={
            "type": "json_object"
        },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

In [31]:
# 함수 리팩토링

def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    당신은 머신러닝/딥러닝/LLM/AI서비스 개발의 전문가로써, 해당 분야의 으뜸가는 면접관입니다.
    매번 그룹사의 인재를 발굴하기 위해 면접 질문/모범 답안을 작성하고 있습니다.
    """

    user_message = f"""

    <<지시사항>>
    - 사용자가 제출한 채용 공고의 내용을 바탕으로 예상 면접 질문과 답변을 작성해 주세요.
    - 하드스킬과 소프트스킬 두 개의 섹션으로 나누어 작성해 주세요.
    - 각 스킬별로 질문/답변을 3개씩 만들어 주세요.

    <<출력 형식>>
    출력은 json 형식으로만 반환되어야 합니다.

    {{
        "hard_skill": [
            {{
                "question": "",
                "answer": ""
            }}
        ],
        "soft_skill": [
            {{
                "question": "",
                "answer": ""
            }}
        ]
    }}

    <<채용 공고>>
    {job_posting}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
            "role": "system",
            "content": [
                {
                "type": "text",
                "text": system_message
                }
            ]
            },
            {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": user_message
                }
            ]
            }
        ],
        response_format={
            "type": "json_object"
        },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

In [32]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>   (MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""
output = job_interview(job_posting)
print(type(output))
print(output)

<class 'str'>
{
    "hard_skill": [
        {
            "question": "Python을 사용하여 머신러닝 인프라를 구축할 때 주로 어떤 패키지를 사용하나요?",
            "answer": "주로 TensorFlow, PyTorch, Scikit-learn과 같은 머신러닝 라이브러리와 함께 Pandas와 NumPy를 데이터 전처리에 사용합니다. 또한, MLflow 같은 툴을 이용해 모델 버전 관리와 배포에도 활용합니다."
        },
        {
            "question": "컨테이너 오케스트레이션 도구인 Kubernetes를 설정 및 관리한 경험에 대해 설명해 주세요.",
            "answer": "저는 Kubernetes 클러스터를 구축하고, 여러 애플리케이션을 Pods로 배포하여 서비스를 운영한 경험이 있습니다. Helm을 이용해 패키징하고, CI/CD 파이프라인과 통합하여 지속적인 배포를 지원했습니다."
        },
        {
            "question": "Jenkins와 ArgoCD를 이용하여 CI/CD 파이프라인을 어떻게 구성하였는지 설명해 주세요.",
            "answer": "Jenkins를 사용하여 코드가 커밋될 때마다 테스트를 실행하고, 성공 시 Docker 이미지를 빌드했습니다. 이후, ArgoCD를 사용하여 Kubernetes 클러스터에 이 이미지를 자동으로 배포하도록 구성하였습니다."
        }
    ],
    "soft_skill": [
        {
            "question": "팀원 간의 기술 지원을 제공할 때 중요한 요소는 무엇이라고 생각하나요?",
            "answer": "팀원들에게 명확하고 간결한 커뮤니케이션이 가장 중요하다고 생각합니다. 또한, 상대방의 이해도를 고려한 피드백 및 질문을 통해 협력하여 문제를 해결하는 것이 필수적입니다.

In [30]:
import json

# 응답 데이터가 json 형식이 아니라면 오류 발생!
data = json.loads(output)
print(type(data))
print(data)

hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']
print(hard_skill_qa)
print(soft_skill_qa)

<class 'dict'>
{'hard_skill': [{'question': 'Python 프로그래밍을 사용하여 머신러닝 인프라를 구축할 때 가장 중요한 고려 사항은 무엇인가요?', 'answer': '머신러닝 인프라를 구축할 때는 코드의 가독성, 재사용성 및 유지보수성을 가장 중요하게 고려해야 합니다. 또한, 성능 최적화와 더불어, 다양한 라이브러리와 프레임워크의 적절한 사용이 필요합니다. 예를 들어, TensorFlow나 PyTorch와 같은 라이브러리를 적절히 사용하여 모델을 쉽게 배포할 수 있도록 설계해야 합니다.'}, {'question': 'Docker와 Kubernetes를 활용하여 머신러닝 모델을 배포한 경험에 대해 이야기해 주실 수 있나요?', 'answer': '네, 저는 Docker를 사용하여 모델을 컨테이너화하고, Kubernetes를 통해 클러스터 환경에서 배포한 경험이 있습니다. 이 과정에서 먼저 Dockerfile을 작성하여 필요 라이브러리와 종속성을 정의했고, 그 후 Helm Chart를 사용하여 Kubernetes에서의 배포 관리를 자동화했습니다. 이로 인해 모델의 업데이트와 확장을 수월하게 수행할 수 있었습니다.'}, {'question': 'CI/CD 파이프라인에서 Jenkins와 Gitlab을 사용하는 경험에 대해 설명해 주시겠어요?', 'answer': '저는 Jenkins를 사용하여 CI/CD 파이프라인을 구성한 경험이 있습니다. 코드가 Gitlab에 푸시될 때마다 Jenkins가 트리거되어 자동으로 빌드와 테스트를 실행하고, 성공하면 프로덕션 환경에 배포하는 흐름을 만들었습니다. 이 과정에서 Jenkinsfile을 사용하여 파이프라인을 코드화하여 가독성과 버전 관리를 용이하게 했습니다.'}], 'soft_skill': [{'question': '팀원과의 협업이 중요한 프로젝트에서 자신의 역할을 어떻게 정의하겠습니까?', 'answer': '팀과의 협업은 성공적인 프로젝트 수행에 필수적입니다. 저는 기술 지원 역할을 맡